In [ ]:
from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import Literal
import pandas as pd

In [10]:
client = genai.Client(
    vertexai=True,
    location='europe-west4'
)

In [12]:
class AccidentVictim(BaseModel):
    age: int | None
    gender: str | None
    occupation: str | None
    injury: str | None
    body_affected: Literal["ears", "eyes", "legs", "arms", "head", "shoulder"] | None
    category: Literal["accident", "crime", "natural_disaster"] | None

In [21]:
accident_descriptions = [
    """
    A 30-year-old man was injured in a car accident. He was driving his car when another car hit him from the side. 
    He was taken to the hospital with a broken leg. 
    The man works as a lawyer.
    """,

    """
    I was walking down the street when I saw a woman fall off her bike. She was taken to the hospital with a broken arm. 
    She was wearing a helmet, so she didn't hit her head. The accident happened on a busy street, so there were many witnesses. 
    To make a living she works as a surgeon.
    """,

    """
    An earthquake hit the city, causing a building to collapse. 
    In an office a 58-year-old employee was injured by falling debris. He got cut wounds
    on his shoulder. His occupation is a teacher.
    """,

    """
    A 10-year-old boy was playing in a park when he fell from a swing. He landed awkwardly on his wrist, 
    resulting in a fracture. He was transported to the local children's hospital for 
    treatment. The boy is a student at the nearby elementary school.
    """,

    """
    A construction worker, aged 45, was injured on a building site. He was operating a 
    forklift when it tipped over, pinning his foot. He sustained severe crush injuries 
    and was airlifted to a regional trauma centre. He is employed as a heavy machinery operator.
    """,

    """
    A car accident left a 30-year-old man with a broken leg. He was hit from the side by another vehicle while driving. The man, a lawyer, was taken to the hospital.
    """,

    """
    While walking down a busy street, a woman fell off her bike. She suffered a broken arm. Thankfully, she was wearing a helmet. The woman, a surgeon, was taken to the hospital by ambulance.
    """,

    """
    Falling debris injured a 58-year-old employee during a building collapse caused by an earthquake. He sustained cut wounds on his shoulder. The man works as a teacher.
    """,

    """
    A fractured wrist was the result of a fall from a swing for a 10-year-old boy. He landed awkwardly while playing in a park and was transported to the local children's hospital. He is a student at the nearby elementary school.
    """,

    """
    Severe crush injuries to his foot required a 45-year-old construction worker to be airlifted to a regional trauma centre. The incident occurred when a forklift he was operating tipped over. He is employed as a heavy machinery operator.
    """,

    """
    Slipping on ice while walking her dog led to a hip fracture for a 65-year-old woman. She was admitted to the hospital for surgery. She is a retired librarian.
    """,

    """
    Burns to his hands and face resulted from a chemical spill during a laboratory experiment, injuring a 22-year-old student. He was treated in the emergency room and released.
    """,

    """
    Chest injuries were sustained by a 50-year-old truck driver in a multi-vehicle collision on the highway, caused by heavy fog. He was hospitalized.
    """,

    """
    A knee injury was suffered by a 16-year-old girl during a high school basketball game, landing awkwardly after a jump shot. She was taken to the hospital for an MRI.
    """,

    """
    Head trauma occurred when a 70-year-old man fell down a flight of stairs at his home. He was taken to the intensive care unit. He is a retired accountant.
    """,

    """
    Severe burns to his arms resulted from a pot of boiling water spilling on a 35-year-old chef working in a restaurant kitchen. He was treated at a burn center.
    """,

    """
    A fractured ankle was the outcome of a fall while trekking in a mountainous area for a 28-year-old hiker. She was rescued by a search and rescue team. She is a freelance graphic designer.
    """,

    """
    Electrocution while working on a power line caused severe burns to a 40-year-old electrician. He was hospitalized and is employed by a local utility company.
    """,

    """
    Puncture wounds to his leg were inflicted by a dog bite on a 12-year-old boy playing in his backyard. He was taken to the hospital for treatment and rabies shots.
    """,

    """
    Multiple fractures were sustained when a tractor overturned, injuring a 55-year-old farmer. He was airlifted to a regional hospital.
    """,

    """
    A back injury occurred when a stack of boxes fell on a 25-year-old warehouse worker. He was taken to the hospital for evaluation and works as a forklift operator.
    """,

    """
    A shoulder dislocation was suffered by a 19-year-old lifeguard while attempting a rescue in the ocean. He was treated at the beach first aid station.
    """,

    """
    A broken arm was the result of a fall from a ladder for a 60-year-old carpenter working on a home renovation project. He was taken to the hospital.
    """,

    """
    A concussion was sustained when a 32-year-old cyclist was hit by a car on a rural road. He was hospitalized and works as a software engineer.
    """,

    """
    Minor injuries were experienced by a 48-year-old pilot during a small plane crash during a training flight. He was rescued.
    """,
]

In [22]:
def extract_accident_victim(accident_description: str) -> AccidentVictim:
    response = client.models.generate_content(
        model='gemini-2.0-flash-001',
        contents=[
            accident_description
        ],
        config=types.GenerateContentConfig(
            temperature=0.2,
            response_mime_type='application/json',
            response_schema=AccidentVictim,
        ),
    )
    accident_victim: AccidentVictim = response.parsed
    return accident_victim

accident_victims = [
    extract_accident_victim(description) for description in accident_descriptions
]
accident_victims

[AccidentVictim(age=30, gender='male', occupation='lawyer', injury='broken leg', body_affected='legs', category='accident'),
 AccidentVictim(age=None, gender='female', occupation='surgeon', injury='broken arm', body_affected='arms', category='accident'),
 AccidentVictim(age=58, gender=None, occupation='teacher', injury='cut wounds', body_affected='shoulder', category='natural_disaster'),
 AccidentVictim(age=10, gender='male', occupation='student', injury='fracture', body_affected='arms', category='accident'),
 AccidentVictim(age=45, gender=None, occupation='heavy machinery operator', injury='severe crush injuries', body_affected='legs', category='accident'),
 AccidentVictim(age=30, gender='male', occupation='lawyer', injury='broken leg', body_affected='legs', category='accident'),
 AccidentVictim(age=None, gender='female', occupation='surgeon', injury='broken arm', body_affected='arms', category='accident'),
 AccidentVictim(age=58, gender=None, occupation='teacher', injury='cut wounds'

In [23]:
pd.DataFrame([victim.model_dump() for victim in accident_victims])

,age,gender,occupation,injury,body_affected,category
0,30.0,male,lawyer,broken leg,legs,accident
1,NaN,female,surgeon,broken arm,arms,accident
2,58.0,None,teacher,cut wounds,shoulder,natural_disaster
3,10.0,male,student,fracture,arms,accident
4,45.0,None,heavy machinery operator,severe crush injuries,legs,accident
5,30.0,male,lawyer,broken leg,legs,accident
6,NaN,female,surgeon,broken arm,arms,accident
7,58.0,None,teacher,cut wounds,shoulder,natural_disaster
8,10.0,male,student,fractured wrist,arms,accident
9,45.0,None,heavy machinery operator,Severe crush injuries,legs,accident
